In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
import geopy.distance
from sklearn.pipeline import Pipeline
from datetime import timedelta 

In [2]:
df = pd.read_csv('nyc bus data.csv')
df = df.drop_duplicates()
# = df.dropna()
df.head()

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,24:06:14,NaN
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,24:00:53,NaN
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,24:03:00,NaN
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN


In [3]:
df.shape

(1048575, 18)

In [4]:
#subset the dataframe to include only the first 100,000 rows, or observations.
#df = df.iloc[:700000, :]
#df.shape

In [5]:
#checking for column types and missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
RecordedAtTime               1048575 non-null object
DirectionRef                 1048575 non-null int64
PublishedLineName            1048575 non-null object
OriginName                   1040112 non-null object
OriginLat                    1040112 non-null float64
OriginLong                   1040112 non-null float64
DestinationName              1048575 non-null object
DestinationLat               1047329 non-null float64
DestinationLong              1047329 non-null float64
VehicleRef                   1048575 non-null object
VehicleLocation.Latitude     1048575 non-null float64
VehicleLocation.Longitude    1048575 non-null float64
NextStopPointName            1047439 non-null object
ArrivalProximityText         1047439 non-null object
DistanceFromStop             1047439 non-null object
ExpectedArrivalTime          931342 non-null object
ScheduledArrivalTime         102290

In [6]:
#filling in missing data 
df['OriginName'].ffill(axis=0, inplace=True)
df['OriginLat'].ffill(axis=0, inplace=True)
df['OriginLong'].ffill(axis=0, inplace=True)
df['DestinationLat'].ffill(axis=0, inplace=True)
df['DestinationLong'].ffill(axis=0, inplace=True)
df['DistanceFromStop'].ffill(axis=0, inplace=True)
df['ArrivalProximityText'].ffill(axis=0, inplace=True)
df['NextStopPointName'].ffill(axis=0, inplace=True)
df['ExpectedArrivalTime'].ffill(axis=0, inplace=True)
df['ScheduledArrivalTime'].ffill(axis=0, inplace=True)


In [7]:
#splitting 'ExpectedArrivalTime' column
expected_split = df['ExpectedArrivalTime'].str.split(expand=True)
print(expected_split.head())

          0     1
0  6/1/2017  0:03
1  6/1/2017  0:03
2  6/1/2017  0:03
3  6/1/2017  0:04
4  6/1/2017  0:03


In [8]:
#creating new feature; the 'expected_date' column
df['expected_date'] = expected_split[0]

In [9]:
#formatting the 'ScheduledArrivalTime'
df['ScheduledArrivalTime'] = df['ScheduledArrivalTime'].apply(lambda x: x[0:2].replace('24', '00')) + df['ScheduledArrivalTime'].apply(lambda x: x[2:])

In [10]:
#examining the 'ScheduledArrivalTime' column
df['ScheduledArrivalTime'].head()

0    00:06:14
1    23:58:02
2    00:00:53
3    00:03:00
4    23:59:38
Name: ScheduledArrivalTime, dtype: object

In [11]:
#creating new feature'Scheduled_datetime' column
df['Scheduled_datetime'] = df['expected_date'] + ' ' + df['ScheduledArrivalTime']

In [12]:
#examining 'Scheduled_datetime' column
df['Scheduled_datetime'].head()

0    6/1/2017 00:06:14
1    6/1/2017 23:58:02
2    6/1/2017 00:00:53
3    6/1/2017 00:03:00
4    6/1/2017 23:59:38
Name: Scheduled_datetime, dtype: object

In [13]:
#converting 'Scheduled_datetime' to datetime object
df['Scheduled_datetime'] = pd.to_datetime(df['Scheduled_datetime'], errors='coerce')

In [14]:
#converting 'Expected_arrival' column to datetime object
df['Expected_datetime'] = pd.to_datetime(df['ExpectedArrivalTime'], errors='coerce')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
RecordedAtTime               1048575 non-null object
DirectionRef                 1048575 non-null int64
PublishedLineName            1048575 non-null object
OriginName                   1048575 non-null object
OriginLat                    1048575 non-null float64
OriginLong                   1048575 non-null float64
DestinationName              1048575 non-null object
DestinationLat               1048575 non-null float64
DestinationLong              1048575 non-null float64
VehicleRef                   1048575 non-null object
VehicleLocation.Latitude     1048575 non-null float64
VehicleLocation.Longitude    1048575 non-null float64
NextStopPointName            1048575 non-null object
ArrivalProximityText         1048575 non-null object
DistanceFromStop             1048575 non-null object
ExpectedArrivalTime          1048575 non-null object
ScheduledArrivalTime         10485

In [16]:
#filling in missing data
df['Scheduled_datetime'].ffill(axis=0, inplace=True)
df['Expected_datetime'].ffill(axis=0, inplace=True)


In [17]:
df.isnull().sum().sum()

1048516

The above null values are in the "Unnamed" column, which I will drop later

In [18]:
#fixing the date value for those datetime entries where the hour is near midnight.
df['Scheduled_datetime'] = df['Scheduled_datetime'].apply(lambda x: x - timedelta(days=1) if str(x)[11:13] == '23' else x)

In [ ]:
#create new feature to record the difference between each bus's scheduled arrival time and expected arrival time.
df['delay'] = df['Expected_datetime'] - df['Scheduled_datetime']

In [ ]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN,6/1/2017,2017-05-31 23:58:02,2017-06-01 00:03:00,00:04:58
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,00:00:53,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00,00:02:07
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,00:03:00,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00,00:01:00
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN,6/1/2017,2017-05-31 23:59:38,2017-06-01 00:03:00,00:03:22


In [ ]:
#creating new feature called 'HourOfDay', to reflect relationship of the time of day and whether or not a bus is on time.
#Certain bus routes may be effected by non controllable conditions such as traffic, which is more of a factor during rush hour
#and accounting for the hour of day may help the model capture that relationship. 
df['Scheduled_HourOfDay'] = df['Scheduled_datetime'].dt.hour

In [ ]:
#creating a new feature "delay_second" to equate 'delay' column in seconds
df['delay_second'] = df['delay'].dt.seconds

In [ ]:
#creating new feature 'delay_minute' to convert the delay_second to minutes
df['delay_minute'] = df['delay_second'] / 60

In [ ]:
#creating new feature delay_days by extracting the day value from 'delay' column
df['delay_days'] = df['delay'].dt.days

In [ ]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,Scheduled_HourOfDay,delay_second,delay_minute,delay_days
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46,0,86206,1436.766667,-1
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,23:58:02,NaN,6/1/2017,2017-05-31 23:58:02,2017-06-01 00:03:00,00:04:58,23,298,4.966667,0
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,00:00:53,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00,00:02:07,0,127,2.116667,0
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,00:03:00,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00,00:01:00,0,60,1.000000,0
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,23:59:38,NaN,6/1/2017,2017-05-31 23:59:38,2017-06-01 00:03:00,00:03:22,23,202,3.366667,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 26 columns):
RecordedAtTime               1048575 non-null object
DirectionRef                 1048575 non-null int64
PublishedLineName            1048575 non-null object
OriginName                   1048575 non-null object
OriginLat                    1048575 non-null float64
OriginLong                   1048575 non-null float64
DestinationName              1048575 non-null object
DestinationLat               1048575 non-null float64
DestinationLong              1048575 non-null float64
VehicleRef                   1048575 non-null object
VehicleLocation.Latitude     1048575 non-null float64
VehicleLocation.Longitude    1048575 non-null float64
NextStopPointName            1048575 non-null object
ArrivalProximityText         1048575 non-null object
DistanceFromStop             1048575 non-null object
ExpectedArrivalTime          1048575 non-null object
ScheduledArrivalTime         10485

# Data Exploration:

#### Exploring dataframe where "delay_days" equals zero.

In [ ]:
#filtering the original dataset
df_zero = df[df['delay_days'] == 0]

In [ ]:
df_zero.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,Scheduled_HourOfDay,delay_second,delay_minute,delay_days
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,23:58:02,NaN,6/1/2017,2017-05-31 23:58:02,2017-06-01 00:03:00,00:04:58,23,298,4.966667,0
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,00:00:53,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00,00:02:07,0,127,2.116667,0
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,00:03:00,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00,00:01:00,0,60,1.000000,0
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,23:59:38,NaN,6/1/2017,2017-05-31 23:59:38,2017-06-01 00:03:00,00:03:22,23,202,3.366667,0
5,6/1/2017 0:03,0,M1,4 AV/E 10 ST,40.731342,-73.990288,HARLEM 147 ST via MADISON,40.821110,-73.935898,NYCT_3831,...,00:02:35,NaN,6/1/2017,2017-06-01 00:02:35,2017-06-01 00:03:00,00:00:25,0,25,0.416667,0


In [ ]:
#value counts of the dates in scheduled datetime
df_zero.Scheduled_datetime.dt.date.value_counts()

2017-06-02    187562
2017-06-01    176890
2017-06-05    161573
2017-06-03    132538
2017-06-04    104490
2017-05-31      1509
Name: Scheduled_datetime, dtype: int64

In [ ]:
#value_counts of the dates in Expected datetime
df_zero.Expected_datetime.dt.date.value_counts()

2017-06-02    187434
2017-06-01    177178
2017-06-05    162137
2017-06-03    132419
2017-06-04    105394
Name: Expected_datetime, dtype: int64

In [ ]:
df_zero.delay.value_counts()

00:01:00    21215
00:00:00    20973
00:02:00    19748
00:03:00    17457
00:04:00    15582
00:05:00    13712
00:06:00    12310
00:07:00    10952
00:08:00     9878
00:09:00     8917
00:10:00     7993
00:11:00     7055
00:12:00     6291
00:13:00     5477
00:14:00     4953
00:15:00     4187
00:16:00     3671
00:17:00     3111
00:18:00     2872
00:19:00     2403
00:12:02     2121
00:20:00     2113
00:21:00     1884
00:06:40     1718
00:22:00     1661
00:22:12     1582
00:36:00     1567
00:23:00     1519
00:24:00     1402
00:00:52     1230
            ...  
01:04:49        1
23:55:07        1
01:03:37        1
03:13:00        1
00:55:18        1
23:25:00        1
23:55:37        1
23:30:47        1
01:09:54        1
01:14:38        1
01:21:54        1
00:54:09        1
00:57:12        1
23:52:43        1
01:34:00        1
00:47:03        1
01:45:00        1
00:53:34        1
23:55:14        1
01:11:58        1
01:20:05        1
23:32:43        1
23:54:23        1
01:10:36        1
23:45:26  

#### Exploring the dataset where "delay_days" equals -1. 

In [ ]:
#filtering the original dataset for only those observations where 'delay_days' equals -1
df_neg1 = df[df['delay_days'] == -1]

In [ ]:
df_neg1.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,Scheduled_HourOfDay,delay_second,delay_minute,delay_days
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46,0,86206,1436.766667,-1
6,6/1/2017 0:03,0,B31,GERRITSEN AV/GERRITSEN BEACH,40.587101,-73.918503,MIDWOOD KINGS HWY STA,40.608433,-73.957100,NYCT_4611,...,00:08:00,NaN,6/1/2017,2017-06-01 00:08:00,2017-06-01 00:03:00,-1 days +23:55:00,0,86100,1435.000000,-1
11,6/1/2017 0:03,1,B1,87 ST/4 Av,40.621925,-74.028496,MANHATTAN BEACH KINGSBORO CC,40.578094,-73.940170,NYCT_7141,...,00:05:55,NaN,6/1/2017,2017-06-01 00:05:55,2017-06-01 00:03:00,-1 days +23:57:05,0,86225,1437.083333,-1
12,6/1/2017 0:03,1,B26,WYCKOFF ST/PALMETTO ST,40.699150,-73.910477,DNTWN BKLYN TILLARY ST via HALSEY,40.693939,-73.990311,NYCT_6503,...,00:05:55,NaN,6/1/2017,2017-06-01 00:05:55,2017-06-01 00:03:00,-1 days +23:57:05,0,86225,1437.083333,-1
13,6/1/2017 0:03,0,Bx39,SOUNDVIEW AV/STEPHENS AV,40.807869,-73.852715,WAKEFIELD 241 ST via WHITE PLS RD,40.903309,-73.849922,NYCT_4723,...,00:04:12,NaN,6/1/2017,2017-06-01 00:04:12,2017-06-01 00:04:00,-1 days +23:59:48,0,86388,1439.800000,-1


In [ ]:
df_neg1.shape

(271111, 26)

In [ ]:
#calculating the time difference for those buses that arrived before time
df_neg1['neg1_minute_diff'] = df_neg1['Expected_datetime'].dt.minute - df_neg1['Scheduled_datetime'].dt.minute

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
#creating a new feature "neg1_minute_diff" to account for the minute differences.
print(df_neg1.neg1_minute_diff.value_counts().head(20))

-1     70133
 0     64714
-2     48293
-3     28850
-4     15888
-5      9125
-6      5552
-7      3716
-8      2547
-9      1955
 58     1789
-10     1599
 57     1587
 59     1478
 56     1330
-11     1191
-12      944
 55      939
-13      712
 54      704
Name: neg1_minute_diff, dtype: int64


In [ ]:
#filtering the df_neg1 dataframe for only those observations where the value of 'neg1_minute_diff' is  greater than -5
df_neg1 = df_neg1[df_neg1['neg1_minute_diff'] > -5]

In [ ]:
#fixing the values of 'delay_minute' column
#df_neg1.loc[(df_neg1['neg1_minute_diff'] <= -5) | (df_neg1['neg1_minute_diff'] > 5), 'delay_minute'] = 6
#df_neg1.loc[(df_neg1['neg1_minute_diff'] >= 0) & (df_neg1['neg1_minute_diff'] < 6), 'delay_minute'] = 0
#df_neg1.loc[(df_neg1['neg1_minute_diff'] <= 0) & (df_neg1['neg1_minute_diff'] > -5), 'delay_minute'] = 0


df_neg1.loc[(df_neg1['neg1_minute_diff'] <= 5), 'delay_minute'] = 0
df_neg1.loc[(df_neg1['neg1_minute_diff'] > 5), 'delay_minute'] = 6

In [ ]:
df_neg1.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,Scheduled_HourOfDay,delay_second,delay_minute,delay_days,neg1_minute_diff
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46,0,86206,0.0,-1,-3
11,6/1/2017 0:03,1,B1,87 ST/4 Av,40.621925,-74.028496,MANHATTAN BEACH KINGSBORO CC,40.578094,-73.940170,NYCT_7141,...,NaN,6/1/2017,2017-06-01 00:05:55,2017-06-01 00:03:00,-1 days +23:57:05,0,86225,0.0,-1,-2
12,6/1/2017 0:03,1,B26,WYCKOFF ST/PALMETTO ST,40.699150,-73.910477,DNTWN BKLYN TILLARY ST via HALSEY,40.693939,-73.990311,NYCT_6503,...,NaN,6/1/2017,2017-06-01 00:05:55,2017-06-01 00:03:00,-1 days +23:57:05,0,86225,0.0,-1,-2
13,6/1/2017 0:03,0,Bx39,SOUNDVIEW AV/STEPHENS AV,40.807869,-73.852715,WAKEFIELD 241 ST via WHITE PLS RD,40.903309,-73.849922,NYCT_4723,...,NaN,6/1/2017,2017-06-01 00:04:12,2017-06-01 00:04:00,-1 days +23:59:48,0,86388,0.0,-1,0
15,6/1/2017 0:03,1,Bx31,VAN CORTLANDT PK E/E 242 ST,40.902660,-73.868378,WESTCHESTER SQ,40.841076,-73.842850,NYCT_274,...,NaN,6/1/2017,2017-06-01 00:05:00,2017-06-01 00:03:00,-1 days +23:58:00,0,86280,0.0,-1,-2


In [ ]:
df_neg1.delay_minute.value_counts()

0.0    227900
6.0     12070
Name: delay_minute, dtype: int64

In [ ]:
#value counts of the dates in scheduled datetime
df_neg1.Scheduled_datetime.dt.date.value_counts()

2017-06-05    61862
2017-06-01    59746
2017-06-02    52953
2017-06-03    34746
2017-06-04    30663
Name: Scheduled_datetime, dtype: int64

In [ ]:
#value counts of the dates in Expected datetime
df_neg1.Expected_datetime.dt.date.value_counts()

2017-06-05    61862
2017-06-01    59746
2017-06-02    52953
2017-06-03    34746
2017-06-04    30663
Name: Expected_datetime, dtype: int64

In [ ]:
#dropping the 'neg1_minute_diff' column from df_neg1
df_neg1 = df_neg1.drop('neg1_minute_diff', 1)

In [ ]:
df_neg1.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,Scheduled_HourOfDay,delay_second,delay_minute,delay_days
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46,0,86206,0.0,-1
11,6/1/2017 0:03,1,B1,87 ST/4 Av,40.621925,-74.028496,MANHATTAN BEACH KINGSBORO CC,40.578094,-73.940170,NYCT_7141,...,00:05:55,NaN,6/1/2017,2017-06-01 00:05:55,2017-06-01 00:03:00,-1 days +23:57:05,0,86225,0.0,-1
12,6/1/2017 0:03,1,B26,WYCKOFF ST/PALMETTO ST,40.699150,-73.910477,DNTWN BKLYN TILLARY ST via HALSEY,40.693939,-73.990311,NYCT_6503,...,00:05:55,NaN,6/1/2017,2017-06-01 00:05:55,2017-06-01 00:03:00,-1 days +23:57:05,0,86225,0.0,-1
13,6/1/2017 0:03,0,Bx39,SOUNDVIEW AV/STEPHENS AV,40.807869,-73.852715,WAKEFIELD 241 ST via WHITE PLS RD,40.903309,-73.849922,NYCT_4723,...,00:04:12,NaN,6/1/2017,2017-06-01 00:04:12,2017-06-01 00:04:00,-1 days +23:59:48,0,86388,0.0,-1
15,6/1/2017 0:03,1,Bx31,VAN CORTLANDT PK E/E 242 ST,40.902660,-73.868378,WESTCHESTER SQ,40.841076,-73.842850,NYCT_274,...,00:05:00,NaN,6/1/2017,2017-06-01 00:05:00,2017-06-01 00:03:00,-1 days +23:58:00,0,86280,0.0,-1


In [ ]:
#Getting the shape of df_zero and df_neg1
print(df_zero.shape)
print()
print(df_neg1.shape)

(764562, 26)

(239970, 26)


In [ ]:
#joining df_zer0 and df_neg1 into one dataframe
df = pd.concat([df_zero, df_neg1], axis=0, ignore_index=True)
#df = df.reset_index()

In [ ]:
#getting the shape of the concatenated dataframe, df
print(df.shape)

(1004532, 26)


In [ ]:
df.delay_days.value_counts()

 0    764562
-1    239970
Name: delay_days, dtype: int64

In [ ]:
#creating output variable
df['on_time'] = np.where((df['delay_minute'] > 5) | (df['delay_minute'] < -1), 0, 1)

In [ ]:
#getting value count for binary output variable
df['on_time'].value_counts()

1    605330
0    399202
Name: on_time, dtype: int64

In [ ]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,Scheduled_HourOfDay,delay_second,delay_minute,delay_days,on_time
0,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,NaN,6/1/2017,2017-05-31 23:58:02,2017-06-01 00:03:00,00:04:58,23,298,4.966667,0,1
1,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00,00:02:07,0,127,2.116667,0,1
2,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00,00:01:00,0,60,1.000000,0,1
3,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,NaN,6/1/2017,2017-05-31 23:59:38,2017-06-01 00:03:00,00:03:22,23,202,3.366667,0,1
4,6/1/2017 0:03,0,M1,4 AV/E 10 ST,40.731342,-73.990288,HARLEM 147 ST via MADISON,40.821110,-73.935898,NYCT_3831,...,NaN,6/1/2017,2017-06-01 00:02:35,2017-06-01 00:03:00,00:00:25,0,25,0.416667,0,1


In [ ]:
#examining column 'Scheduled_HourOfDay', to see which hour of the day most routes are scheduled for.
df.Scheduled_HourOfDay.value_counts()

17    73328
16    71589
15    69304
8     67299
14    64149
18    63953
7     62918
9     58182
13    57548
19    53496
12    53202
11    51528
10    51238
20    43303
6     40413
21    28609
22    22993
5     19840
0     18270
4      8373
1      7813
23     6111
2      5743
3      5330
Name: Scheduled_HourOfDay, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004532 entries, 0 to 1004531
Data columns (total 27 columns):
RecordedAtTime               1004532 non-null object
DirectionRef                 1004532 non-null int64
PublishedLineName            1004532 non-null object
OriginName                   1004532 non-null object
OriginLat                    1004532 non-null float64
OriginLong                   1004532 non-null float64
DestinationName              1004532 non-null object
DestinationLat               1004532 non-null float64
DestinationLong              1004532 non-null float64
VehicleRef                   1004532 non-null object
VehicleLocation.Latitude     1004532 non-null float64
VehicleLocation.Longitude    1004532 non-null float64
NextStopPointName            1004532 non-null object
ArrivalProximityText         1004532 non-null object
DistanceFromStop             1004532 non-null object
ExpectedArrivalTime          1004532 non-null object
ScheduledArrivalTime         10045

In [ ]:
df["source"] = [(i, j) for i, j in zip(df["OriginLat"], df["OriginLong"])]

df["destination"] = [(i, j) for i, j in zip(df["DestinationLat"], df["DestinationLong"])]

df["dist"] = [geopy.distance.distance(i, j).km for i, j in zip(df["source"], df["destination"])]

In [ ]:
df['Vehicle_location2'] = [(i,j) for i, j in zip(df['VehicleLocation.Latitude'], df['VehicleLocation.Longitude'])]
df["VehicleToDestination"] = [geopy.distance.distance(i, j).km for i, j in zip(df["Vehicle_location2"], df["destination"])]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#exploring the 'OriginName' column
df.OriginName.describe()

It's important to know exactly how many different origin points we have in our data; by examining the "OriginName" column, we see that there are 605 unique origin points.  Next I will subset the data based on the two different values of the output variable, "on_time", to see if there is any relation between the Origin point of the bus and whether its on time or not.

In [ ]:
#creating two subsets of the dataframe; new_df_0 represents the subset with buses not on time, and new_df_1
# represents the subset with buses on time.
df_0 = df[df['on_time'] == 0]
df_1 = df[df['on_time'] == 1]


In [ ]:
#exploring the 'OriginName' column for those buses that did not make it to their destinations on time.
df_0.OriginName.describe()

In [ ]:
#exploring the 'OriginName' column for those buses that made it on time.
df_1.OriginName.describe()

In [ ]:
#exploring the DestinationName column
print(new_df_0.DestinationName.describe())
print()
print(new_df_1.DestinationName.describe())

Here I'm assuming the column "OriginName" to represent a bus stop, defined by the street intersection at which that bus stop is located at, from where each bus's route started, for each recorded observation in the data.

After subsetting the dataframe based on the binary values of the output variable, 'on_time', I noticed that the two subsets of the dataframe had similar number of unique "OriginNames; 587 out of 605 for the subset representing buses that were delayed and 604 out of 605 for the subset of data representing buses that were on time.  This suggests that both of the separate subsets had observations from almost all of the same bus stops, and that no one particular route was either always on time or always delayed.  Further analysis of the "OriginName" column below may help us see a pattern between the starting point (OriginName) of a bus's route and whether that bus was on time or not.

In [ ]:
#examing the top ten value_counts of dataframe new_df_0.
df_1.OriginName.value_counts().head(20)

In [ ]:
#examining the top ten value_counts of dataframe new_df_0
df_0.OriginName.value_counts().head(20)

In [ ]:
print(type(df_0.OriginName.value_counts()))

After examining the two lists of "OriginName" value counts above, I noticed that the two lists from the separate subsets of the data share alot of the same bus stop names.  Bus stops (OriginName) such as E 126 ST/2 AV, ELTINGVILLE/TRANSIT CENTER, MERRICK BL/ARCHER AV, FLATBUSH AV/KINGS PLAZA, W 179 ST/BROADWAY, and TEARDROP/LAYOVER were involved with alot of delays but also alot of buses that made it to their destination on time.  Considering that this data is regarding New York City, this may suggest that their is another underlying factor causing delays, such as high traffic on the roads due to hour of day, or seasonal/special events at or near some of the specific bus stops that I mentioned here. 

Aside from this, it should be noted that of the 20 Origin names in the second list of value counts (which represents those OriginNames that were related to those buses being delayed) 11 are unique; that is they are not in the top 20 list of value counts for origin names that were related to those buses that were on time.  Names such as W FARMS RD/WESTCHESTER AV, BROADWAY/ISHAM ST, ASTOR PL/3 AV, AND 41 RD/MAIN ST, just to name a few.

To account for some of the other factors, such as traffic, that may be a leading cause of delays, I added the 'Scheduled_HourOfDay' column in the data.

In [ ]:
#making two scatter plots to show the origin locations of buses on time, and not on time, using origin latitude and origin
#longitude

#setting figure size
plt.figure(figsize=(15,10))

#making a plot figure that will have one row with two plots in each row
plt.subplot(1, 2, 1)
plt.scatter(x=df_0.OriginLat, y=df_0.OriginLong, color='blue')
plt.ylabel('Values')
plt.title('Origin Locations for buses Not On Time')

plt.subplot(1, 2, 2)
plt.scatter(x=df_1.OriginLat, y=df_1.OriginLong, color='blue')
plt.ylabel('Values')
plt.title('Origin Locations for buses On Time')
plt.show()

In [ ]:
#making two scatter plots to show the origin locations of buses on time, and not on time, using origin latitude and origin
#longitude

#setting figure size
plt.figure(figsize=(15,10))

#making a plot figure that will have one row with two plots in each row
plt.subplot(1, 2, 1)
plt.scatter(x=df_0.DestinationLat, y=df_0.DestinationLong, color='blue')
plt.ylabel('Values')
plt.title('Destination Locations for buses Not On Time')

plt.subplot(1, 2, 2)
plt.scatter(x=df_1.DestinationLat, y=df_1.DestinationLong, color='blue')
plt.ylabel('Values')
plt.title('Destination Locations for buses On Time')
plt.show()

The two sets of scatterplot show the origin locations and destination locations using latitude and longitude for all buses, on time and those not on time, from the dataset.  As you can see, they are near identical for both scenarios, which further states that the bus route in of itself was not the sole factor for causing a bus to be on time to its destination or delayed.

In [ ]:
df.columns

In [ ]:
#dropping some columns and splitting the data and creating the initial train and test splits.
X = df.drop(['RecordedAtTime', 'ArrivalProximityText', 'Unnamed: 17', 'PublishedLineName', 'VehicleRef', 
                 'NextStopPointName', 'ExpectedArrivalTime', 'ScheduledArrivalTime', 'on_time', 'OriginName', 
                 'DestinationName','source', 'destination', 'Vehicle_location2', 'DistanceFromStop', 'Scheduled_datetime', 
                 'Expected_datetime', 'delay', 'delay_second', 'delay_minute', 'delay_days', 'source', 
                 'destination', 'expected_date'], 1)

#getting dummy values for categorical data, and creating dataframe of the independent (predictor) values
#XD_df = pd.get_dummies(X, columns=['OriginName', 'DestinationName'])


Y = df['on_time']



In [ ]:
#examining the dataframe with the predictor variables
X.head(10)

In [ ]:
#checking shape of X
X.shape

In [ ]:
#checking which column has NaN value
X.isnull().sum().sum()

In [93]:
#checking the shape of outcome variable Y.
Y.shape

(1004532,)

In [94]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=.25)
print(test_X.shape)
print(test_Y.shape)

#checking for any NaN values in our data.
#print(np.isnan(train_Y).any())
#print(np.isnan(train_X).any().sum())
#print(np.isnan(test_Y).any())
#print(np.isnan(test_X).any().sum())

(251133, 10)
(251133,)


In [95]:
# perform grid search to find the optimal parameters for our decision tree model
dtree = tree.DecisionTreeClassifier(random_state=2)
tree_param_grid = {'max_depth':[4,6,10], 'min_samples_split':[6,8,10,15], 'max_features':[2,4,6,8], 
                   'criterion':['gini', 'entropy']}
dtree_grid = GridSearchCV(dtree, tree_param_grid, cv=5, verbose=3)
dtree_grid.fit(train_X, train_Y)
print('Best parameters for decision tree:', dtree_grid.best_params_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] criterion=gini, max_depth=4, max_features=2, min_samples_split=6 
[CV]  criterion=gini, max_depth=4, max_features=2, min_samples_split=6, score=0.6382200690204407, total=   1.1s
[CV] criterion=gini, max_depth=4, max_features=2, min_samples_split=6 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV]  criterion=gini, max_depth=4, max_features=2, min_samples_split=6, score=0.6379811521104327, total=   1.1s
[CV] criterion=gini, max_depth=4, max_features=2, min_samples_split=6 


...

[CV]  criterion=entropy, max_depth=6, max_features=4, min_samples_split=15, score=0.6373615434134816, total=   2.4s
[CV] criterion=entropy, max_depth=6, max_features=6, min_samples_split=6 
[CV]  criterion=entropy, max_depth=6, max_features=6, min_samples_split=6, score=0.6475643748340855, total=   3.5s
[CV] criterion=entropy, max_depth=6, max_features=6, min_samples_split=6 
[CV]  criterion=entropy, max_depth=6, max_features=6, min_samples_split=6, score=0.6449362888239979, total=   3.4s
[CV] criterion=entropy, max_depth=6, max_features=6, min_samples_split=6 
[CV]  criterion=entropy, max_depth=6, max_features=6, min_samples_split=6, score=0.6440602601539687, total=   3.5s
[CV] criterion=entropy, max_depth=6, max_features=6, min_samples_split=6 
[CV]  criterion=entropy, max_depth=6, max_features=6, min_samples_split=6, score=0.6399190337138306, total=   3.9s
[CV] criterion=entropy, max_depth=6, max_features=6, min_samples_split=6 
[CV]  criterion=entropy, max_depth=6, max_features=6, 

[CV]  criterion=entropy, max_depth=10, max_features=2, min_samples_split=6, score=0.656391027342713, total=   2.7s
[CV] criterion=entropy, max_depth=10, max_features=2, min_samples_split=6 
[CV]  criterion=entropy, max_depth=10, max_features=2, min_samples_split=6, score=0.6625209883261768, total=   2.4s
[CV] criterion=entropy, max_depth=10, max_features=2, min_samples_split=8 
[CV]  criterion=entropy, max_depth=10, max_features=2, min_samples_split=8, score=0.6590589328378019, total=   2.5s
[CV] criterion=entropy, max_depth=10, max_features=2, min_samples_split=8 
[CV]  criterion=entropy, max_depth=10, max_features=2, min_samples_split=8, score=0.6598221396336608, total=   2.5s
[CV] criterion=entropy, max_depth=10, max_features=2, min_samples_split=8 
[CV]  criterion=entropy, max_depth=10, max_features=2, min_samples_split=8, score=0.6522564374834086, total=   2.6s
[CV] criterion=entropy, max_depth=10, max_features=2, min_samples_split=8 
[CV]  criterion=entropy, max_depth=10, max_fea

[CV]  criterion=entropy, max_depth=10, max_features=6, min_samples_split=8, score=0.665808335545527, total=   5.8s
[CV] criterion=entropy, max_depth=10, max_features=6, min_samples_split=8 
[CV]  criterion=entropy, max_depth=10, max_features=6, min_samples_split=8, score=0.6635386249004512, total=   5.8s
[CV] criterion=entropy, max_depth=10, max_features=6, min_samples_split=8 
[CV]  criterion=entropy, max_depth=10, max_features=6, min_samples_split=8, score=0.668728431112291, total=   6.2s
[CV] criterion=entropy, max_depth=10, max_features=6, min_samples_split=8 
[CV]  criterion=entropy, max_depth=10, max_features=6, min_samples_split=8, score=0.6656468386437393, total=   5.7s
[CV] criterion=entropy, max_depth=10, max_features=6, min_samples_split=10 
[CV]  criterion=entropy, max_depth=10, max_features=6, min_samples_split=10, score=0.6665848155030528, total=   6.0s
[CV] criterion=entropy, max_depth=10, max_features=6, min_samples_split=10 
[CV]  criterion=entropy, max_depth=10, max_f

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed: 28.2min finished


Best parameters for decision tree: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 8, 'min_samples_split': 10}


In [96]:
print('Best parameters for decision tree:', dtree_grid.best_params_)
print('Best score for decision tree: ', dtree_grid.best_score_)

Best parameters for decision tree: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 8, 'min_samples_split': 10}
Best score for decision tree:  0.6701349484137887


In [97]:
# Create the first model by Initializing and training our tree before any additional preprocessing
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=8,
    max_depth=10,
    min_samples_split=10
)

#set start time
start_time_tree = datetime.datetime.now()

#train our tree
decision_tree.fit(train_X, train_Y)

# Classify, storing the result in a new variable.
y_pred_dtree = decision_tree.predict(test_X)

#get accuracy score from our model
print('Accuracy score for decision tree Model: ' + str(metrics.accuracy_score(test_Y, y_pred_dtree)))

#calculate end time
end_time_tree = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time_tree - start_time_tree))
print()

#get accuracy score of decision tree
tree_score = cross_val_score(decision_tree, X, Y, cv=10, scoring='accuracy')
print(tree_score)
print('Average cross validated score from our decision tree model is: ' + str(np.mean(tree_score)))

Accuracy score for decision tree Model: 0.6686934811434578

Duration: 0:00:09.918549

[0.65029765 0.68637386 0.57828039 0.65770062 0.6336197  0.64322619
 0.63897544 0.70676834 0.61296328 0.66048799]
Average cross validated score from our decision tree model is: 0.6468693451947565


In [ ]:
kbest = SelectKBest(f_classif)
pipeline = Pipeline([('kbest', kbest), ('lr', LogisticRegression())])
gridcv_k_log = GridSearchCV(pipeline, {'kbest__k': [1,2,3,4], 'lr__C': np.logspace(-10, 10, 5)}, verbose=3)
gridcv_k_log.fit(train_X, train_Y)

In [ ]:
print('Best parameters for k_log:', gridcv_k_log.best_params_)
print('Best score for k_log: ', gridcv_k_log.best_score_)

# Logistic Regression

In [ ]:
#initialize and train our model
LogReg = LogisticRegression(C=1e9, random_state=12)

#set start time
start_time4 = datetime.datetime.now()

#fit and train the model
LogReg.fit(train_X, train_Y)

# Classify, storing the result in a new variable.
y_pred_LogReg = LogReg.predict(test_X)

#get accuracy score from our model
print('Accuracy score for Logistic regression Model: ' + str(metrics.accuracy_score(test_Y, y_pred_LogReg)))

#calculate end time
end_time4 = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time4 - start_time4))
print()

#get average accuracy score using cross validation
LogReg_cvs = cross_val_score(LogReg, X, Y, cv=10, scoring='accuracy')
print(LogReg_cvs)

print('Average cross validated score from our log regression model is: ' + str(np.mean(LogReg_cvs)))

# PCA

In [ ]:
# Normalize the data so that all variables have a mean of 0 and standard deviation
# o

X_scaled = StandardScaler().fit_transform(X)

In [ ]:
#Apply PCA

pca = PCA(.95)
X_pca = pca.fit_transform(X_scaled)

comp_no = pca.n_components_
comp_var_ratio = pca.explained_variance_ratio_

print(comp_no)
print(comp_var_ratio)
#print(p_components)

In [ ]:
#pca data to dataframe
pca_df = pd.DataFrame(data=X_pca)
print(pca_df.head())
print(pca_df.shape)

In [ ]:
#After applying PCA, I split the data into a second set of train and test groups

train_X_pca, test_X_pca, train_Y_pca, test_Y_pca = train_test_split(X_pca, Y, test_size=.3, random_state=20)

# Random Forest

In [ ]:
# perform grid search to find the optimal parameters for our random forest model, after applying PCA
rforest = ensemble.RandomForestClassifier(random_state=20)
forest_param_grid = {'n_estimators':[50,100], 'max_depth':[4,6,8,10], 'min_samples_split':[6,8,10,12], 
                     'max_features':[2,4,6], 'criterion':['entropy', 'gini']}
rforest_grid = GridSearchCV(rforest, forest_param_grid, cv=10, verbose=3)
rforest_grid.fit(train_X, train_Y)
print('Best parameters for decision tree:', rforest_grid.best_params_)

In [ ]:
print('Best score for Random Forest:', rforest_grid.best_score_)

In [ ]:
#initialize and train our random forest model with PCA
rand_forest_pca = ensemble.RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=10, max_features=6,
                                              min_samples_split=10, random_state=300)

#set start time
start_time_rforest = datetime.datetime.now()

rand_forest_pca.fit(train_X_pca, train_Y_pca)

# Classify, storing the result in a new variable.
y_pred_rforest_pca = rand_forest_pca.predict(test_X_pca)

#get accuracy score from our model
print('Accuracy score for Random Forest Model: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_rforest)))

#calculate end time
end_time_rforest = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time_rforest - start_time_rforest))
print()

#get average accuracy score of our random forest through cross validation
rforest_pca_cvs = cross_val_score(rand_forest_pca, X_pca, Y, cv=10, scoring='accuracy')
print(rforest_pca_cvs)
print('Average cross validated score from our Random Forest Model is: ' + str(np.mean(rforest_pca_cvs)))

In [ ]:
# create confusion matrix for random forest and PCA
confusion_matrix(test_Y_pca, y_pred_rforest_pca)

In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_rforest_pca = LogReg.predict_proba(test_X_pca)
# keep probabilities for the positive outcome only
probs_rforest_pca = probs_rforest_pca[:, 1]
# calculate AUC
rforest_auc = roc_auc_score(test_Y_pca, probs_rforest_pca)
print('AUC Score: %.3f' % rforest_auc)
# calculate roc curve
fpr_rforest_pca, tpr_rforest_pca, thresholds_rforest_pca = roc_curve(test_Y_pca, probs_rforest_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_rforest_pca, tpr_rforest_pca, marker='.')
# show the plot
plt.show()

In [ ]:
# perform classification report of the logistic regression model
class_names = ['Class_0', 'Class_1']
rforest_report = classification_report(test_Y_pca, y_pred_rforest_pca, target_names=class_names)
print(rforest_report)

# Gradient Boosting

In [ ]:
#Run gridsearchcv to find the best parameters for gradient boosting using PCA
gboost_param_grid = {'n_estimators':[200,400,600], 'max_depth':[2,4,6], 'min_samples_leaf': [3, 5, 9], 
                     'learning_rate': [.05, .02, .01], 'subsample': [.7, .8, .9]}
gb_model = ensemble.GradientBoostingClassifier(random_state=67)

gb_grid = GridSearchCV(gb_model, gboost_param_grid, cv=5)
gb_grid.fit(train_X_pca, train_Y_pca)
print('Best parameters for Gradient Boost Model are:', gb_grid.best_params_)

In [ ]:
print('Best parameters for Gradient Boost Model are:', gb_grid.best_params_)
print('Best score for Gradient Boost Model are:', gb_grid.best_score_)

In [ ]:
# Initialize and fit the model using PCA
Grad_boost_pca = ensemble.GradientBoostingClassifier(learning_rate=0.02, max_depth=2, min_samples_leaf=5, 
                                                 n_estimators=200, subsample=0.7, random_state=67)

#set start time
start_time_gb = datetime.datetime.now()

#fit and train the model
Grad_boost_pca.fit(train_X_pca, train_Y_pca)

# Classify, storing the result in a new variable.
y_pred_Gboost_pca = Grad_boost.predict(test_X_pca)

#get accuracy score from our model
print('Accuracy score for Gradient Boost Model: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_Gboost_pca)))

#calculate end time
end_time_gb = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time_gb - start_time_gb))
print()

#get average accuracy score using cross validation
Grad_boost_cvs_pca = cross_val_score(Grad_boost_pca, X_pca, Y, cv=5, scoring='accuracy')
print(Grad_boost_cvs_pca)

print('Average cross validated score from our Gradient Boosting model is: ' + str(np.mean(Grad_boost_cvs_pca)))

In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_Gboost_pca = Grad_boost_pca.predict_proba(test_X_pca)
# keep probabilities for the positive outcome only
probs_Gboost_pca = probs_Gboost_pca[:, 1]
# calculate AUC
Gboost_auc = roc_auc_score(test_Y_pca, probs_Gboost_pca)
print('AUC Score: %.3f' % Gboost_auc)
# calculate roc curve
fpr_Gboost_pca, tpr_Gboost_pca, thresholds_Gboost_pca = roc_curve(test_Y_pca, probs_Gboost_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_Gboost_pca, tpr_Gboost_pca, marker='.')
# show the plot
plt.show()

print()
# perform classification report of the Gradient Boosting model
class_names = ['Class_0', 'Class_1']
Gboost_report = classification_report(test_Y_pca, y_pred_Gboost_pca, target_names=class_names)
print(Gboost_report)
print()
#confusion matrix
print('Confusion Matrix:')
confusion_matrix(test_Y_pca, y_pred_Gboost_pca)


# Logistic Regression

In [ ]:
#initialize and train our model after PCA
LogReg_pca = LogisticRegression(C=1e9, random_state=12)

#set start time
start_time4 = datetime.datetime.now()

#fit and train the model
LogReg_pca.fit(train_X_pca, train_Y_pca)

# Classify, storing the result in a new variable.
y_pred_LogReg_pca = LogReg_pca.predict(test_X_pca)

#get accuracy score from our model
print('Accuracy score for Logistic regression Model after pca: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_LogReg_pca)))

#calculate end time
end_time4 = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time4 - start_time4))
print()

#get average accuracy score using cross validation
LogReg_cvs_pca = cross_val_score(LogReg_pca, X_pca, Y, cv=10, scoring='accuracy')
print(LogReg_cvs_pca)

print('Average cross validated score from our log regression model is: ' + str(np.mean(LogReg_cvs_pca)))

In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_LogReg_pca = LogReg_pca.predict_proba(test_X)
# keep probabilities for the positive outcome only
probs_LogReg_pca = probs_LogReg_pca[:, 1]
# calculate AUC
LogReg_pca_auc = roc_auc_score(test_Y_pca, probs_LogReg_pca)
print('AUC Score: %.3f' % LogReg_pca_auc)
# calculate roc curve
fpr_LogReg_pca, tpr_LogReg_pca, thresholds_LogReg_pca = roc_curve(test_Y_pca, probs_LogReg_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_LogReg_pca, tpr_LogReg_pca, marker='.')
# show the plot
plt.show()
print()

# perform classification report of the logistic regression model
class_names = ['Class_0', 'Class_1']
LogReg_report_pca = classification_report(test_Y_pca, y_pred_LogReg_pca, target_names=class_names)
print(LogReg_report_pca)
print()

# create confusion matrix
confusion_matrix(test_Y_pca, y_pred_LogReg_pca)

# KNN

In [ ]:
# perform grid search to find the optimal parameters for our KNN model using full data.
KNN_model = KNeighborsClassifier()
KNN_params = {'n_neighbors':[5,10,15], 'weights':['uniform', 'distance']}
KNN_grid = GridSearchCV(KNN_model, KNN_params, cv=5)
KNN_grid.fit(train_X_pca, train_Y_pca)
print('Best parameters for data:', KNN_grid.best_params_)

In [ ]:
#initialize and train our model using the training set
KNN_pca = KNeighborsClassifier(n_neighbors=5, weights='distance')

#set start time
starttime_knn = datetime.datetime.now()

#fit and train the model
KNN_pca.fit(train_X_pca,train_Y_pca)

#Make predictions using test_X, and store results in new variable
y_pred_KNN_pca = KNN_pca.predict(test_X_pca)

print('Accuracy score for KNN Model: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_KNN_pca)))

#calculate end time
endtime_knn = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(endtime_knn - starttime_knn))
print()


KNN_cvs_pca = cross_val_score(KNN_pca, X_pca, Y, cv=5, scoring='accuracy')
print(KNN_cvs_pca)
print('Average cross validated score from our KNN model is: ' + str(np.mean(KNN_cvs_pca)))


In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_KNN_pca = KNN_pca.predict_proba(test_X_pca)
# keep probabilities for the positive outcome only
probs_KNN_pca = probs_KNN_pca[:, 1]
# calculate AUC
KNN_pca_auc = roc_auc_score(test_Y_pca, probs_KNN_pca)
print('AUC Score: %.3f' % KNN_pca_auc)
# calculate roc curve
fpr_KNN_pca, tpr_KNN_pca, thresholds_KNN_pca = roc_curve(test_Y_pca, probs_KNN_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_KNN_pca, tpr_KNN_pca, marker='.')
# show the plot
plt.show()

# perform classification report of the KNN model
class_names = ['Class_0', 'Class_1']
KNN_report_pca = classification_report(test_Y_pca, y_pred_KNN_pca, target_names=class_names)
print(KNN_report_pca)

# confusion Matrix
confusion_matrix(test_Y_pca, y_pred_KNN_pca)


# SelectKBest

In [ ]:
kbest = SelectKBest(f_classif)
pipeline = Pipeline([('kbest', kbest), ('lr', LogisticRegression())])
grid_search = GridSearchCV(pipeline, {'kbest__k': [1,2,3,4], 'lr__C': np.logspace(-10, 10, 5)})
grid_search.fit(train_X_pca, train_Y_pca)

In [ ]:
#performing GridsearchCV to get the best parameters for SelectKBest.
#Kbest = SelectKBest().fit_transform(X, Y)
#K_params = {'k':[1,2,3,4,5,6], 'score_func':[chi2, f_classif]}
#Kgrid = GridSearchCV(Kbest, K_params, cv=5, scoring='accuracy')
#Kgrid.fit(X, Y)
#print('Best parameters for SelectKBest:', Kgrid.best_params_)

In [ ]:
#applying selectKBest.
X_skb = SelectKBest(f_classif, k=6).fit_transform(X, Y)
print(X_skb.shape)

In [ ]:
#After applying SelectKBest, I split the data into a third set of train and test groups
train_X_skb, test_X_skb, train_Y_skb, test_Y_skb = train_test_split(X_skb, Y, test_size=.3, random_state=20)